# Find local minima on the scanned potential energy surface by greedy algorithm

Necessary packages

In [ ]:
import os
import yaml
from itertools import product

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.patches import Rectangle

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
def get_step_to_adjacent_points(fsize, dim=2, cutoff=np.inf):
    one_d_points = list(range(- fsize, fsize + 1))
    var_combinations = product(*[one_d_points] * dim)
    for points in var_combinations:
        dist = np.linalg.norm(np.array(points))
        if dist <= cutoff:
            yield points

def get_energy(coord, energies):
    try:
        return energies[coord]
    except IndexError:
        new_coord = tuple(x if x < energies.shape[i] else
                          x - energies.shape[i] for i, x in enumerate(coord))
        return energies[new_coord]

def compare_to_adjacent_point(coord, energies, unchecked_points, filters):
        
    # each element is a coordinate
    new_coords = [tuple(x + var_x for x, var_x in zip(coord, var))
                  for var in filters]

    # Get the energies of adjacent points
    energies = [get_energy(new_coord, energies) for new_coord in new_coords]

    # Sorted
    energies, new_coords = zip(*sorted(zip(energies, new_coords)))

    # Find the current point index and points that has higher energy than this point
    # Will be removed from unchecked points list
    cur_point_ind = new_coords.index(coord)
    for new_coord in new_coords[cur_point_ind:]:
        try:
            unchecked_points.remove(new_coord)
        except ValueError:
            # ValueError if coord_min is not in unchecked_points
            pass
    return new_coords[0]  


def search_for_a_minimum(coord, energies, unchecked_points, filters):
    while True:
        next_point = compare_to_adjacent_point(coord, energies,
                                               unchecked_points, filters)
        next_point = tuple(x if x >= 0 else energies.shape[i] + x
                           for i, x in enumerate(next_point))
        if next_point == coord:
            return coord
        elif next_point not in unchecked_points:
            return
        else:
            coord = next_point

            
def search_minimum(energies, fsize, cutoff=np.inf):
    minimum = []
    
    dim = len(energies.shape)
    filters = list(get_step_to_adjacent_points(fsize, dim, cutoff))

    oned_points = [list(range(energies.shape[i])) for i in range(dim)]
    unchecked_points = list(product(*oned_points))

    while True:
        if not unchecked_points:
            break 
        coord = unchecked_points[np.random.randint(len(unchecked_points))]
        new_min = search_for_a_minimum(coord, energies,
                                       unchecked_points, filters)
        if new_min:
            minimum.append(new_min)
    return minimum

## 1. Load energies from the yaml file

In [ ]:
path = '/Users/xiaorui/Downloads/en_imipramine_4_oo_2d_scan/en_imipramine_4_oo_3_1_2_4_n_2_4_11_21_coord.yml'
steps = 45

In [ ]:
with open(path) as file:
        bookkeep = yaml.load(file, Loader=yaml.FullLoader)
        
energies = np.zeros((steps, steps))

# Get the maximum energy and set NaNs based on the maximum energy
max_energy = max([e for e in bookkeep.values() if not np.isnan(e)])

for key in bookkeep.keys():
    i, j = key
    if np.isnan(bookkeep[key]):
        energies[i, j] = 0.99 * max_energy if max_energy < 0 else 1.01 * max_energy
    else:
        energies[i, j] = bookkeep[key]

# Rescale the energy based on the lowest energy
# This will not change the result of search but make detailed view more clear
energies = energies - np.min(energies)

## 2. Search for the local minimum
Optional arguments:
- fsize (int): a number indicate the filter size. For example, if `fsize = 1`, the minimal point found will be smaller than the adjacent point with indexes plus/minus 1.

In [ ]:
minimum_points = search_minimum(energies, fsize=2)

## 3. Plot the energy surface with minimum identified

In [ ]:
fig_size = (28, 20); annot=True  # detailed view
fig_size = (5, 4); annot=False  # overlook view


In [ ]:
f, ax = plt.subplots(figsize=fig_size)

# Remove NaNs from the surface
mask = np.zeros_like(energies)
for key in bookkeep.keys():
    i, j = key
    if np.isnan(bookkeep[key]):
        mask[i, j] = 1

# Plot as an heatmap by Seaborn
ax = sns.heatmap(energies, vmin=0, vmax=0.2, cmap="YlGnBu", annot=annot, annot_kws={"fontsize":8}, mask=mask)

# Identified the minimum by red rectangle patches
for point in minimum_points:
    # In the heatmap, the first index is for the y-axis
    # while in the pyplot the first index is for the x-axis
    # therefore, for displaying, we need to invert the axis
    if energies[point[0], point[1]] < 0.5:
        ax.add_patch(Rectangle(point[::-1], 1, 1, fill=False, edgecolor='red', lw=2))

plt.show()